In [ ]:
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D, UpSampling2D, Input

import cv2
import os
import numpy as np

In [ ]:
import tensorflow as tf
devices = tf.config.experimental.get_visible_devices('GPU')
tf.config.experimental.set_memory_growth(device=devices[0], enable = True)

In [ ]:
# dataset from keras
(X_train, y_train),(X_test, y_test) = mnist.load_data()

In [ ]:
# load your dataset here
def load_data(path):
    images = [os.path.join(path, i) for i in os.listdir(path) if i.endswith(".jpg") or i .endswith(".png")]
    images = [cv2.imread(image_path) for image_path in images]
    images = [cv2.cvtColor(image, cv2.COLOR_BGR2RGB) for image in images]
    images = [cv2.resize(image, (28, 28)) for image in images]
    return np.array(images)

#X_train = load_data('H:/Datasets/Face dataset/WIDER_val/images/0--Parade')

In [ ]:
X_train.shape

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X_train[0], cmap = 'gray')

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28,28,1)
X_test = X_test.reshape(X_test.shape[0], 28,28,1)
X_train.shape

# Model

In [ ]:
input_layer = Input(shape = (28,28,1))

# encoder
x = Conv2D(16, (3,3), activation = 'relu', padding = 'same')(input_layer)
x = MaxPool2D(pool_size = (2,2), padding = 'same')(x)
x = Conv2D(8, (3,3), activation = 'relu', padding = 'same')(x)
x = MaxPool2D(pool_size = (2,2), padding = 'same')(x)
x = Conv2D(8, (3,3), activation = 'relu', padding = 'same')(x)
encoded = MaxPool2D(pool_size = (2,2), padding = 'same')(x)

# decoder
x = Conv2D(8, (3,3), activation = 'relu', padding = 'same')(encoded)
x = UpSampling2D((2,2))(x)
x = Conv2D(8, (3,3), activation = 'relu', padding = 'same')(x)
x = UpSampling2D((2,2))(x)
x = Conv2D(16, (3,3), activation = 'relu')(x)
x = UpSampling2D((2,2))(x)
decoded = Conv2D(1, (3,3), activation = 'relu', padding = 'same')(x)

In [ ]:
autoencoder = Model(input_layer, decoded)
autoencoder.summary()

In [ ]:
# compile
autoencoder.compile(loss = 'binary_crossentropy', optimizer = 'adam')

In [ ]:
history = autoencoder.fit(X_train, X_train, epochs=50, batch_size = 128, validation_data=(X_test,X_test))

In [ ]:
autoencoder.summary()

In [ ]:
encoder = Model(input_layer, encoded)
encoder.summary()

In [ ]:
decoder_layer = Input(shape = (4,4,8))

decoder = autoencoder.layers[7](decoder_layer)

for layer in autoencoder.layers[8:]:
    decoder = layer(decoder)

decoder = Model(decoder_layer, decoder, name = 'Decoder')
decoder.summary()

In [ ]:
# encoder images for testing
encoded_images = encoder.predict(X_test, verbose = 1)
encoded_images.shape

In [ ]:
decoded_images = decoder.predict(encoded_images, verbose=1)
decoded_images.shape

In [ ]:
decoded_images = decoded_images.reshape(decoded_images.shape[0], 28,28)
decoded_images.shape

In [ ]:
plt.imshow(decoded_images[1], cmap = 'gray')

In [ ]:
X_test = X_test.reshape(X_test.shape[0], 28,28)
plt.imshow(X_test[1], cmap = 'gray')